In [1]:
%cd -q ..

import numpy as np
import pandas as pd

from model.helpers import load_params
from model.inpatients import InpatientsModel
from model.outpatients import OutpatientsModel
from model.aae import AaEModel

pd.set_option("display.precision", 1)

In [ ]:
import logging
logging.basicConfig(
    format="%(asctime)s %(levelname)-8s %(message)s",
    level=logging.INFO,
    datefmt="%Y-%m-%d %H:%M:%S",
)

In [ ]:
params = load_params("sample_params.json")

# Helper Methods

Below are some helper methods for turning the results into useful Pandas `DataFrames`.

In [ ]:
def run_model(model):
    params = load_params("sample_params.json")

    return {
        k0: pd.DataFrame([
            { **dict(k1), "value": v1 }
            for k1, v1 in v0.items()
        ])
        for k0, v0 in model(params, "data").go(0).items()
    }

In [ ]:
def default_pod_measure_pivot(agg):
    default_results = (
        agg["default"]
        .groupby(["pod", "measure"], as_index=False)
        .agg({"value": sum})
        .pivot(index=["pod"], columns="measure")
        .fillna(0)
    )
    default_results.loc["total"] = default_results.sum()

    return default_results

In [ ]:
def step_counts_simple(agg):
    step_counts = (
        agg["step_counts"]
        .drop(columns=["strategy", "activity_type"])
        .groupby(["change_factor", "measure"], as_index=False)
        .sum()
        .pivot(index="change_factor", columns="measure")
        .loc[agg["step_counts"]["change_factor"].unique()]
    )
    step_counts.loc["total"] = step_counts.sum()

    return step_counts

# Running the models

## Inpatients

Running the principal model run (model run 0) for the Inpatients data

In [ ]:
ip_agg = run_model(InpatientsModel)

### Aggregate Results

The available aggregations for inpatients are:

In [ ]:
list(ip_agg.keys())

['default',
 'sex+age_group',
 'sex+tretspef',
 'bed_occupancy',
 'theatres_available',
 'step_counts']

We can view the default aggregations, summarising to pod/measure

In [ ]:
default_pod_measure_pivot(ip_agg)

value                                 
measure                   admissions attendances   beddays procedures
pod                                                                  
ip_elective_admission         7002.0         0.0   30127.0     3940.0
ip_elective_daycase          54276.0         0.0   54276.0    29473.0
ip_maternity_admission        7052.0         0.0   30868.0     3746.0
ip_non-elective_admission    44413.0         0.0  162258.0     6008.0
op_procedure                     0.0       929.0       0.0        0.0
total                       112743.0       929.0  277529.0    43167.0

### Estimate of effects

In [ ]:
step_counts_simple(ip_agg)

value          
measure                    admissions   beddays
change_factor                                  
baseline                      96864.0  308769.0
demographic_adjustment        22355.1   75440.7
health_status_adjustment      -3403.7  -12679.5
expat                          -210.4    -746.4
repat                          2335.2    8004.1
waiting_list_adjustment         165.2     249.5
baseline_adjustment               0.0       0.0
non-demographic_adjustment    11926.1   40845.7
activity_avoidance           -16359.6  -74080.0
efficiencies                   -929.0  -68274.0
total                        112743.0  277529.0

## Outpatients

Running the principal model run (model run 0) for the Outpatients data

In [ ]:
op_agg = run_model(OutpatientsModel)

### Aggregate Results

The available aggregations for outpatients are:

In [ ]:
list(op_agg.keys())

['default', 'sex+age_group', 'sex+tretspef', 'step_counts']

We can view the default aggregations, summarising to pod/measure

In [ ]:
default_pod_measure_pivot(op_agg)

value                 
measure      attendances tele_attendances
pod                                      
op_first          123544            14245
op_follow-up      290824            36083
op_procedure           0            40887
total             414368            91215

### Estimate of effects

In [ ]:
step_counts_simple(op_agg)

value                 
measure                  attendances tele_attendances
change_factor                                        
baseline                    440289.0          14655.0
demographic_adjustment       85841.0           3206.3
health_status_adjustment    -10372.7           -352.1
expat                        -1351.5            -46.2
repat                        10228.3            344.7
waiting_list_adjustment       7027.7            238.9
baseline_adjustment              0.0              0.0
activity_avoidance          -42682.8          -1442.7
efficiencies                -74611.0          74611.0
total                       414368.0          91215.0

## Accident and Emergency

Running the principal model run (model run 0) for the A&E data

In [ ]:
ae_agg = run_model(AaEModel)

### Aggregate Results

The available aggregations for outpatients are:

In [ ]:
list(ae_agg.keys())

['default', 'sex+age_group', 'step_counts']

We can view the default aggregations, summarising to pod/measure

In [ ]:
default_pod_measure_pivot(ae_agg)

value         
measure     ambulance  walk-in
pod                           
aae_type-01   37107.0  66615.0
aae_type-02       5.0    923.0
aae_type-03     218.0  16038.0
aae_type-04       0.0    836.0
total         37330.0  84412.0

### Estimate of effects

In [ ]:
step_counts_simple(ae_agg)

,value
measure,arrivals
change_factor,
baseline,116368.0
demographic_adjustment,16109.3
health_status_adjustment,-1192.2
expat,-2690.6
repat,2636.8
baseline_adjustment,0.0
activity_avoidance,-9489.3
total,121742.0
